In [2]:
import requests
from bs4 import BeautifulSoup, Tag, NavigableString
from urllib.parse import urljoin, urlparse, urldefrag
import time
import re
import wikipediaapi
from fake_useragent import UserAgent
import wikipediaapi
import urllib.parse

import time
import re 


In [5]:
ua = UserAgent()
USER_AGENT = ua.random

REQUEST_TIMEOUT = 10  
CRAWL_DELAY = 1 
# SESSION = requests.Session()
headers = {'User-Agent': ua.random}
# SESSION.headers.update({"User-Agent": ua.random})
# MAX_PAGES_PER_DOMAIN = 100 

In [4]:
def get_webpage_html(url):
    try:
        response = requests.get(url, headers=headers, timeout=REQUEST_TIMEOUT)
        response.raise_for_status()
        return response.text 
    except requests.exceptions.RequestException as e:
        print(f"General request error fetching HTML from '{url}': {e}")
        return None


In [6]:
get_webpage_html('https://www.cell.com/heliyon/home')

General request error fetching HTML from 'https://www.cell.com/heliyon/home': 403 Client Error: Forbidden for url: https://www.cell.com/heliyon/home


In [4]:
def get_href_list(html_content):
    base_url = 'https://dsvh.gov.vn'
    
    soup = BeautifulSoup(html_content, 'lxml')
    table = soup.find('table', class_='Table')
    href_list = []
    tags = table.find_all('a', href= True)
    
    for tag in tags:
        relative_href = tag['href']
        abs_href = urljoin(base_url,relative_href)
        
        href_list.append(abs_href)
    return href_list

In [6]:
def extract_article_content_simple_footer_break(page_html_content):

    if not page_html_content:
        return None, None

    soup = BeautifulSoup(page_html_content, 'lxml')

    # Extract title
    h1_title_tag = soup.find('h1', class_='page-title')
    page_title = h1_title_tag.get_text(strip=True) if h1_title_tag else "No Title Found"

    # Extract paragraphs and stop at footer
    p_tags = soup.find_all('p')
    
    collected_paragraphs_text = []
    for p in p_tags:
        text = p.get_text(separator=' ', strip=True)
        
        if not text: 
            continue


        if "địa chỉ:" in text.lower():
            break 
        collected_paragraphs_text.append(text)
        
    main_content = "\n\n".join(collected_paragraphs_text).strip()

    return page_title, main_content

In [ ]:

start_url = "https://dsvh.gov.vn/danh-muc-di-tich-quoc-gia-dac-biet-1752"


print(f"Fetching initial page: {start_url}")
initial_html = get_webpage_html(start_url)

all_articles_data = []

if initial_html:
    print("\nExtracting links from the table...")
    links_to_scrape = get_href_list(initial_html)

In [14]:
if links_to_scrape:
        print(f"Found {len(links_to_scrape)} links to scrape.")
        
        for i, article_url in enumerate(links_to_scrape[29:]):
            print(f"\n--- Processing link {i+1}/{len(links_to_scrape)}: {article_url} ---")


            article_html = get_webpage_html(article_url)
            
            if article_html:
                title, content = extract_article_content_simple_footer_break(article_html)
                if title and content: 
                    print(f"Title: {title}")
                    all_articles_data.append({
                        'url': article_url,
                        'title': title,
                        'content': content
                    })
                elif title: # Content might be empty if footer was immediate
                    print(f"Title: {title}")
                    print("Content appears to be empty or only footer was found.")
                    all_articles_data.append({ # Still save title if found
                        'url': article_url,
                        'title': title,
                        'content': "" 
                    })
                else:
                    print("Could not extract title/content properly.")
            else:
                print(f"Failed to fetch HTML for {article_url}")
            
            print(f"Waiting for {CRAWL_DELAY} second(s)...")
            time.sleep(CRAWL_DELAY)
else:
    print("No links found in the table to scrape.")

print(f"\n\n--- Scraping Complete ---")
print(f"Successfully processed data from {len(all_articles_data)} articles.")

# Example: Print titles and first 50 chars of content for verification
if all_articles_data:
    print("\n--- Sample of Scraped Data ---")
    for i, article in enumerate(all_articles_data[:3]): # Print first 3
        print(f"\nArticle {i+1}:")
        print(f"  URL: {article['url']}")
        print(f"  Title: {article['title']}")
        print(f"  Content Preview: {article['content'][:150]}...")

Found 91 links to scrape.

--- Processing link 1/91: https://dsvh.gov.vn/di-tich-kien-truc-nghe-thuat-chua-keo-huyen-vu-thu-tinh-thai-binh-2964 ---
Title: Di tích kiến trúc nghệ thuật chùa Keo (huyện Vũ Thư, tỉnh Thái Bình)
Waiting for 1 second(s)...

--- Processing link 2/91: https://dsvh.gov.vn/di-tich-khao-co-va-kien-truc-nghe-thuat-oc-eo-ba-the-2965 ---
Title: Di tích khảo cổ và kiến trúc nghệ thuật Óc Eo - Ba Thê
Waiting for 1 second(s)...

--- Processing link 3/91: https://dsvh.gov.vn/di-tich-khao-co-va-kien-truc-nghe-thuat-go-thap-2966 ---
Title: Di tích khảo cổ và kiến trúc nghệ thuật Gò Tháp
Waiting for 1 second(s)...

--- Processing link 4/91: https://dsvh.gov.vn/danh-lam-thang-canh-ho-ba-be-2967 ---
Title: Danh lam thắng cảnh Hồ Ba Bể
Waiting for 1 second(s)...

--- Processing link 5/91: https://dsvh.gov.vn/danh-lam-thang-canh-vuon-quoc-gia-cat-tien-2968 ---
Title: Danh lam thắng cảnh Vườn Quốc gia Cát Tiên
Waiting for 1 second(s)...

--- Processing link 6/91: https://dsvh.g

In [173]:
def get_a_tags(page_html):
    base_url = 'https://dsvh.gov.vn'
    soup = BeautifulSoup(page_html,'lxml')
    content = soup.find(class_ = 'page-content')
    a_tags = content.find_all('a', href=True)
    href_list = []
    for tag in a_tags:
        if tag.find('img'):
            continue
        
        relative_href = tag['href']
        if "_pageindex=" in relative_href.lower():
            continue
        abs_href = urljoin(base_url,relative_href)
        
        href_list.append(abs_href)
    return href_list
    

In [180]:
list_htmls=[]
for link in range(1,16):
    url = 'https://dsvh.gov.vn/bao-vat-quoc-gia-128?_pageIndex=' + str(link)
    list_htmls.append(get_webpage_html(url))

In [181]:
len(list_htmls)

15

In [182]:
href_links = []

In [183]:
for html in list_htmls:
    first_list = get_a_tags(html)
    first_list = list(first_list)
    for link in first_list:
        href_links.append(link)

In [184]:
len(href_links)

141

In [185]:
all_articles_data = []

if href_links:
        print(f"Found {len(href_links)} links to scrape.")
        
        for i, article_url in enumerate(href_links):
            print(f"\n--- Processing link {i+1}/{len(href_links)}: {article_url} ---")


            article_html = get_webpage_html(article_url)
            
            if article_html:
                title, content = extract_article_content_simple_footer_break(article_html)
                if title and content: 
                    print(f"Title: {title}")
                    all_articles_data.append({
                        'url': article_url,
                        'title': title,
                        'content': content
                    })
                elif title: # Content might be empty if footer was immediate
                    print(f"Title: {title}")
                    print("Content appears to be empty or only footer was found.")
                    all_articles_data.append({ # Still save title if found
                        'url': article_url,
                        'title': title,
                        'content': "" 
                    })
                else:
                    print("Could not extract title/content properly.")
            else:
                print(f"Failed to fetch HTML for {article_url}")
            
            print(f"Waiting for {CRAWL_DELAY} second(s)...")
            time.sleep(CRAWL_DELAY)
else:
    print("No links found in the table to scrape.")

print(f"\n\n--- Scraping Complete ---")

if all_articles_data:
    print("\n--- Sample of Scraped Data ---")
    for i, article in enumerate(all_articles_data[:3]): 
        print(f"\nArticle {i+1}:")
        print(f"  URL: {article['url']}")
        print(f"  Title: {article['title']}")
        print(f"  Content Preview: {article['content'][:150]}...")

Found 141 links to scrape.

--- Processing link 1/141: https://dsvh.gov.vn/la-de-trang-tri-chim-phuong-dat-nung-thoi-ly-hoang-thanh-thang-long-22206 ---
Title: Lá đề trang trí chim Phượng đất nung thời Lý, Hoàng thành Thăng Long
Waiting for 1 second(s)...

--- Processing link 2/141: https://dsvh.gov.vn/hai-tuong-su-tu-da-thanh-do-ban-binh-dinh-22207 ---
Title: Hai tượng sư tử đá thành Đồ Bàn, Bình Định
Waiting for 1 second(s)...

--- Processing link 3/141: https://dsvh.gov.vn/binh-gom-hoa-nau-hai-phong-22208 ---
Title: Bình gốm hoa nâu, Hải Phòng
Waiting for 1 second(s)...

--- Processing link 4/141: https://dsvh.gov.vn/chum-gom-hoa-nau-hiep-an-thoi-tran-hai-duong-22209 ---
Title: Chum gốm hoa nâu Hiệp An thời Trần, Hải Dương
Waiting for 1 second(s)...

--- Processing link 5/141: https://dsvh.gov.vn/mo-hinh-dat-nung-kien-truc-thoi-le-so-hoang-thanh-thang-long-22174 ---
Title: Mô hình đất nung kiến trúc thời Lê Sơ, Hoàng thành Thăng Long
Waiting for 1 second(s)...

--- Processing link 6

In [186]:
import json

output_json_file = "bao_vat_quoc_gia.json"

if all_articles_data:
    try:
        with open(output_json_file, 'w', encoding='utf-8') as f:
            json.dump(all_articles_data, f, ensure_ascii=False, indent=4)
        print(f"\nSaved data for {len(all_articles_data)} articles to '{output_json_file}'")
    except IOError as e:
        print(f"Error saving data to JSON file: {e}")
else:
    print("No data to save.")


Saved data for 141 articles to 'bao_vat_quoc_gia.json'


In [19]:
soup = BeautifulSoup(test,'lxml')
content = soup.find(class_ = 'page-content')

In [21]:
a_tags = content.find_all('a', href=True)

In [25]:
a_tags[:10]

[<a href="den-xam-tinh-nam-dinh-22212" title="Đền Xám, tỉnh Nam Định">Đền Xám, tỉnh Nam Định</a>,
 <a href="chua-boi-khe-thanh-pho-ha-noi-22201" title="Chùa Bối Khê, Thành phố Hà Nội">Chùa Bối Khê, Thành phố Hà Nội</a>,
 <a href="quan-the-thuong-cang-van-don-tinh-quang-ninh-22179" title="Quần thể Thương cảng Vân Đồn, tỉnh Quảng Ninh">Quần thể Thương cảng Vân Đồn, tỉnh Quảng Ninh</a>,
 <a href="dinh-tra-co-tinh-quang-ninh-22180" title="Đình Trà Cổ, tỉnh Quảng Ninh">Đình Trà Cổ, tỉnh Quảng Ninh</a>,
 <a href="di-tich-khoi-nghia-truong-dinh-tinh-tien-giang-22181" title="Di tích Khởi nghĩa Trương Định, tỉnh Tiền Giang">Di tích Khởi nghĩa Trương Định, tỉnh Tiền Giang</a>,
 <a href="hang-xom-trai-va-mai-da-lang-vanh-tinh-hoa-binh-22172" title="Hang Xóm Trại và mái đá Làng Vành, tỉnh Hòa Bình">Hang Xóm Trại và mái đá Làng Vành, tỉnh Hòa Bình</a>,
 <a href="thap-vinh-hung-tinh-bac-lieu-22173" title="Tháp Vĩnh Hưng, tỉnh Bạc Liêu">Tháp Vĩnh Hưng, tỉnh Bạc Liêu</a>,
 <a href="di-tich-lich-su-chi